### Prompt

You have been given some order information that contains a list of orders with SKUs and related quantities.  Additionally, you have a data set with the total inventory for each SKU.

You have been tasked with figuring out how to consume all of the inventory with the least amount of orders.  In other words, you have been asked to fill the largest orders for each SKU in descending order.

Additionally, only complete orders may be fulfilled.  For example, if an order calls for 4 units of a SKU, and only 3 are left in inventory, you can't fulfill the order.

In [1]:
# importing lib

import pandas as pd

import os

In [3]:
# importing data

cwd = os.getcwd()

file_name1 = 'orders.csv'

file_name2 = 'supply.csv'

file_path_1 = os.path.join(cwd, file_name1)

file_path_2 = os.path.join(cwd, file_name2)

df_orders = pd.read_csv(file_path_1)
df_supply = pd.read_csv(file_path_2)

In [4]:
df_orders.head()

,OrderID,Order SKU,Quantity
0,1,1,2
1,2,3,8
2,3,6,7
3,4,7,7
4,5,8,6


In [5]:
df_supply.head()

,Quantity,SKU
0,15,1
1,180,2
2,159,3
3,184,4
4,52,5


In [6]:
# joining both datasets on SKU

df_merge = df_orders.merge(df_supply, left_on = 'Order SKU', right_on = 'SKU')

df_merge.head()

,OrderID,Order SKU,Quantity_x,Quantity_y,SKU
0,1,1,2,15,1
1,8,1,7,15,1
2,28,1,5,15,1
3,35,1,2,15,1
4,44,1,4,15,1


In [9]:
df_merge = df_merge.rename(columns = {'Quantity_x':'Quantity' , 'Quantity_y' : 'Inventory'})

df_merge.head()

,OrderID,Order SKU,Quantity,Inventory,SKU
0,1,1,2,15,1
1,8,1,7,15,1
2,28,1,5,15,1
3,35,1,2,15,1
4,44,1,4,15,1


In [11]:
# sorting data

df_merge.sort_values(by = ['Quantity', 'Order SKU'], ascending = [False, True], inplace=True)

In [12]:
df_merge.head()

,OrderID,Order SKU,Quantity,Inventory,SKU
23,248,1,8,15,1
24,261,1,8,15,1
35,373,1,8,15,1
36,389,1,8,15,1
46,480,1,8,15,1


In [19]:
df_merge.dtypes

OrderID      int64
Order SKU    int64
Quantity     int64
Inventory    int64
SKU          int64
dtype: object

In [68]:
# finding remaining from inventory

# creating function

def multi_row(sku_group):
    
    remaining_list = []
    
    previous_remaining = None
    
    for index, row in sku_group.iterrows():
        
        if pd.isna(previous_remaining):
            remaining = row['Inventory'] - row['Quantity']
            remaining_list.append(remaining)
            
        elif remaining_list[-1] - row['Quantity'] < 0:
            remaining = remaining_list[-1]
            remaining_list.append(remaining)
        
        elif remaining_list[-1] - row['Quantity'] == 0:
             remaining_list.append(0)
        
        else:
            remaining = remaining_list[-1] - row['Quantity']
            remaining_list.append(remaining)
            
        previous_remaining = remaining
    
    sku_group['remaining'] = remaining_list
    
    # sku_group = sku_group.reset_index(level = 0, drop = True).reset_index()
    
    return sku_group



In [72]:
# applying function

df_remaining = df_merge.groupby('Order SKU').apply(multi_row).reset_index(drop = True)

df_remaining.head()

,OrderID,Order SKU,Quantity,Inventory,SKU,remaining
0,248,1,8,15,1,7
1,261,1,8,15,1,7
2,373,1,8,15,1,7
3,389,1,8,15,1,7
4,480,1,8,15,1,7


In [74]:
# returning unique across order sku, inventory, remaining

df_drop = df_remaining.drop_duplicates(subset = ['Order SKU', 'Inventory', 'remaining'])

df_drop

,OrderID,Order SKU,Quantity,Inventory,SKU,remaining
0,248,1,8,15,1,7
8,8,1,7,15,1,0
64,113,2,8,180,2,172
65,145,2,8,180,2,164
66,209,2,8,180,2,156
...,...,...,...,...,...,...
625,680,11,8,95,11,23
626,171,11,7,95,11,16
627,178,11,7,95,11,9
628,208,11,7,95,11,2


In [77]:
# sorting df_drop

df_drop.sort_values(by = ['Order SKU', 'Quantity', 'remaining'], ascending = [True, False, False]).reset_index(drop = True)

,OrderID,Order SKU,Quantity,Inventory,SKU,remaining
0,248,1,8,15,1,7
1,8,1,7,15,1,0
2,113,2,8,180,2,172
3,145,2,8,180,2,164
4,209,2,8,180,2,156
...,...,...,...,...,...,...
192,680,11,8,95,11,23
193,171,11,7,95,11,16
194,178,11,7,95,11,9
195,208,11,7,95,11,2
